In [1]:
import numpy as np
import pandas as pd
import time

## READ THE DATA

In [2]:
def SplitFile(fileLines):
    g = []
    for line in fileLines:
        if line == '':
            yield g
            g = []
        else:
            g.append(line)
    yield g

In [3]:
fileName = 'input.txt'
fileLines = open(fileName).read().splitlines()
rules, messages = list(SplitFile(fileLines))

## PART 1

In [4]:
indexToRuleDictionary = {}

for rule in rules:
    ruleIndex, ruleCondition = rule.split(": ")
    indexToRuleDictionary[int(ruleIndex)] = ruleCondition

In [5]:
def AppendToRuleDictionary(dictionary, key, element):
    if key in dictionary:
        dictionary[key].append(element)
    else:
        dictionary[key] = element

In [6]:
## This one decodes any rule
def DecodeRule(indexToRuleDictionary, processedRuleDictionary, ruleString):
    ## Get index of pipe, or -1 if there is none
    splitIndex = next((index for index, element in enumerate(ruleString) if element == '|'), -1)
    
    ## Case 1: rule is a single character ("a", or "b")
    if ((splitIndex == -1) and (ruleString[0] == '"')):
        return [ruleString[1:-1]]
    
    ## Case 2: rule is an index or a series of indexes with no pipes ('4 1 5', '3')
    if ((splitIndex == -1) and (ruleString[0] != '"')):
        ruleList = [int(element) for element in ruleString.split(' ')]
        decodedRuleList = ['']
        for ruleIndex in ruleList:
            newDecodedRuleList = DecodeRuleByIndex(indexToRuleDictionary, processedRuleDictionary, ruleIndex)
            decodedRuleList = [str(before) + str(after) for before in decodedRuleList for after in newDecodedRuleList]
        return decodedRuleList
    
    ## Case: rule is two series of indexes split by a pipe ('4 5 | 5 4')
    leftRule = ruleString[:splitIndex-1]
    rightRule = ruleString[splitIndex+2:]
    lstLeft = DecodeRule(indexToRuleDictionary, processedRuleDictionary, leftRule)
    lstRight = DecodeRule(indexToRuleDictionary, processedRuleDictionary, rightRule)
    return lstLeft + lstRight

## This one decodes any indexed rule, updating the processed rule dictionary
def DecodeRuleByIndex(indexToRuleDictionary, processedRuleDictionary, indexToDecode):
    if indexToDecode in processedRuleDictionary:
        return processedRuleDictionary[indexToDecode]
    
    ruleString = indexToRuleDictionary[indexToDecode]
    decodedRule = DecodeRule(indexToRuleDictionary, processedRuleDictionary, ruleString)
    AppendToRuleDictionary(processedRuleDictionary, indexToDecode, decodedRule)
    return decodedRule

In [7]:
dicProcess = {}
tic = time.perf_counter()
DecodeRuleByIndex(indexToRuleDictionary, dicProcess, 0)

matchCounter = 0

for message in messages:
    if message in dicProcess[0]:
        matchCounter += 1
toc = time.perf_counter()
        
print('Number of messages that match rule 0:', matchCounter)
print(f"Total found in {toc - tic:0.4f} seconds.")

Number of messages that match rule 0: 104
Total found in 11.0933 seconds.
